In [3]:
import logging
import threading
import time

import numpy

import samlab.interactive
import samlab.timeseries

logging.basicConfig(level=logging.DEBUG)

In [26]:
stop = samlab.interactive.Stop()

def random_walk(key, count, mean, variance, period, writer):
    for i in range(count):
        if stop.triggered:
            break
        writer.add_scalar(key, i, numpy.random.normal(mean, variance))
        time.sleep(period)

with samlab.timeseries.Writer("samlab") as writer:
    stop = samlab.interactive.Stop()
    
    threads = []
    threads.append(threading.Thread(target=random_walk, args=("foo/reward", 60, 0, 0.1, 0.5, writer)))
    threads.append(threading.Thread(target=random_walk, args=("bar/reward", 60, 0, 0.1, 0.5, writer)))

    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()

In [5]:
import samlab.database
db, fs = samlab.database.connect()

In [20]:
keys_series = list(db.timeseries.aggregate([{"$group": {"_id": {"key": "$key", "series": "$series"}}}, {"$sort":{"_id": 1}}]))
keys_series

[{'_id': {'key': 'samlab/bar/reward', 'series': '2019-10-10T23-32-57'}},
 {'_id': {'key': 'samlab/bar/reward', 'series': '2019-10-10T23-36-22'}},
 {'_id': {'key': 'samlab/foo/reward', 'series': '2019-10-10T23-32-57'}},
 {'_id': {'key': 'samlab/foo/reward', 'series': '2019-10-10T23-36-22'}}]

In [21]:
import collections
keys = collections.defaultdict(list)
for item in keys_series:
    keys[item["_id"]["key"]].append(item["_id"]["series"])
keys

defaultdict(list,
            {'samlab/bar/reward': ['2019-10-10T23-32-57',
              '2019-10-10T23-36-22'],
             'samlab/foo/reward': ['2019-10-10T23-32-57',
              '2019-10-10T23-36-22']})

In [24]:
results = []
for key, series in keys.items():
    results.append({"key": key, "series": series})
results

[{'key': 'samlab/bar/reward',
  'series': ['2019-10-10T23-32-57', '2019-10-10T23-36-22']},
 {'key': 'samlab/foo/reward',
  'series': ['2019-10-10T23-32-57', '2019-10-10T23-36-22']}]

In [7]:
sorted(db.timeseries.distinct("key"))

['samlab/bar/reward', 'samlab/foo/reward']